In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
import torch,cv2,re
import spacy
from spacy.lang.en import English
import pickle
from fastai.text import*
import numpy as np
import json
import csv

This says we will use GPU number 2

In [3]:
torch.cuda.set_device(2)

# START OF MEDICAL STUFF

Opens and reads in the file with all of the medical reports

In [4]:
fh = open('/home/hoc2003/trove-data/trove-studies.json')
studies_string = fh.read()
fh.close()
studies_string[0:100]

'{ "_id" : { "$oid" : "55ff25239ad446c56ba8b321" }, "hl7_json_history" : [], "finalized_date" : null,'

From each medical report we take only the section called finalized_report and put it into a new list while keeping track of the number of files that do not have a finalized report

In [5]:
report_array = []
temp = None
counter=0
for study in studies_string.split('\n'):
    try:
        temp = json.loads(study)
        t=temp['finalized_report']
        if t != None:
            report_array.append(t)
        #report_array.append(temp['finalized_report'])
    except:
        counter=counter+1

We are only interested in PA and LAT reports and AP reports that we then put in seperate lists based on the ID codes

In [6]:
lat_array = []
front_array = []
counter = 0
for x in report_array:
    if re.search(r'DXCHPALAT', x) != None:
        lat_array.append(x)
    if re.search(r'DXMCH1V', x) != None:
        front_array.append(x)

Function that returns a list of all the impressions, a list of reports that didn't have impressions, and the number of reports that didn't have impressions

In [7]:
def get_impressions(arr):
    lst=[]
    other=[]
    counter=0
    for x in arr:
        if re.search(r'IMPRESSION\:(.*?)Prepared By',x)!= None:
            y=re.search(r'IMPRESSION\:(.*?)Prepared By',x).group(1)
            z=y.replace('|',' ').strip()
            lst.append(re.sub(' +',' ',z))
        elif re.search(r'Impression\:(.*?)Prepared By',x) !=None:
            y=re.search(r'Impression\:(.*?)Prepared By',x).group(1)
            z=y.replace('|',' ').strip()
            lst.append(re.sub(' +',' ',z))
        elif re.search(r'Impression(.*?)Prepared By',x) != None:
            y=re.search(r'Impression(.*?)Prepared By',x).group(1)
            z=y.replace('|',' ').strip()
            lst.append(re.sub(' +',' ',z))
        elif re.search(r'IMPRESSION(.*?)Prepared By',x) != None:
            y=re.search(r'IMPRESSION(.*?)Prepared By',x).group(1)
            z=y.replace('|',' ').strip()
            lst.append(re.sub(' +',' ',z))
        else:
            counter=counter+1
            other.append(x)
    return lst,other,counter

Function that returns a lost of all the findings, a list of reports that didn't have findings, and the number of reports that didn't have findings

In [ ]:
def get_findings(arr):
    lst=[]
    other=[]
    counter=0
    for x in arr:
        if re.search(r'Findings\:(.*?)Impression',x)!= None:
            y=re.search(r'Findings\:(.*?)Impression',x).group(1)
            z=y.replace('|',' ').strip()
            lst.append(re.sub(' +',' ',z))
        elif re.search(r'Findings \:(.*?)Impression',x)!= None:
            y=re.search(r'Findings \:(.*?)Impression',x).group(1)
            z=y.replace('|',' ').strip()
            lst.append(re.sub(' +',' ',z))
        elif re.search(r'FINDINGS\:(.*?)IMPRESSION',x) != None:
            y=re.search(r'FINDINGS\:(.*?)IMPRESSION',x).group(1)
            z=y.replace('|',' ').strip()
            lst.append(re.sub(' +',' ',z))
        elif re.search(r'Findings\:(.*?)IMPRESSION',x) != None:
            y=re.search(r'Findings\:(.*?)IMPRESSION',x).group(1)
            z=y.replace('|',' ').strip()
            lst.append(re.sub(' +',' ',z))
        elif re.search(r'FINDINGS/IMPRESSION\:(.*?)Prepared By',x) != None:
            y=re.search(r'FINDINGS/IMPRESSION\:(.*?)Prepared By',x).group(1)
            z=y.replace('|',' ').strip()
            lst.append(re.sub(' +',' ',z))
        elif re.search(r'FINDINGS / IMPRESSION\:(.*?)Prepared By',x) != None:
            y=re.search(r'FINDINGS / IMPRESSION\:(.*?)Prepared By',x).group(1)
            z=y.replace('|',' ').strip()
            lst.append(re.sub(' +',' ',z))
        elif re.search(r'Findings/impression\:(.*?)Prepared By',x) != None:
            y=re.search(r'Findings/impression\:(.*?)Prepared By',x).group(1)
            z=y.replace('|',' ').strip()
            lst.append(re.sub(' +',' ',z))
        elif re.search(r'Findings/Impression\:(.*?)Prepared By',x) != None:
            y=re.search(r'Findings/Impression\:(.*?)Prepared By',x).group(1)
            z=y.replace('|',' ').strip()
            lst.append(re.sub(' +',' ',z))
        elif re.search(r'Findings and Impression\:(.*?)Prepared By',x) != None:
            y=re.search(r'Findings and Impression\:(.*?)Prepared By',x).group(1)
            z=y.replace('|',' ').strip()
            lst.append(re.sub(' +',' ',z))
        else:
            counter=counter+1
            other.append(x)
    return lst,other,counter

The next 4 cells respectivly represent the PA and LAT impression, the PA impression, the PA and LAT findings, and then the AP findings

In [8]:
lat_im, lat_o, lic = get_impressions(lat_array)

In [ ]:
front_im, front_o, fic = get_impressions(front_array)

In [ ]:
lat_fin, lat_fo, lfc = get_findings(lat_array)

In [ ]:
front_fin, front_fo, ffc = get_findings(front_array)

Function that returns the origional string where the first character is lowercase

In [9]:
def first_lower(s):
   if len(s) == 0:
      return s
   else:
      return s[0].lower() + s[1:]

In [ ]:
Function that returns the origional string where the first character is uppercase

In [10]:
def first_upper(s):
   if len(s) == 0:
      return s
   else:
      return s[0].upper() + s[1:]

Reads in the dictionary and removes any quotation marks

In [11]:
reader = csv.reader(open('rad-dict.csv', 'r'))
d = {}
for row in reader:
   k, v = row
   d[k.replace('"','')]=v.replace('"','')

A function that takes in a string and a dictionary and for each key in the dictionary it checks to see if it is in the string and if it is it will replace that key with its value in the corresponding format (uppercase vs lowercase)

In [12]:
def replace_all(text, dic):
    for i, j in dic.items():
        if re.search(r'\b' + i + r'\b', text):
            text = text.replace(i, first_lower(j))
        elif re.search(r'\b' + first_upper(i) + r'\b', text):
            text = text.replace(first_upper(i), first_upper(j))
    return text

A function that takes in a list and creates a new list after applying the replace_all method to the original list

In [13]:
def mod_report(arr):
    mod=[]
    for re in arr:
        mod.append(replace_all(re, d))
    return mod

Applying mod_report to the PA and LAT impression

In [15]:
new_lat_im = mod_report(lat_im)

Zipping and unzipping lists, qs contains the tuples of original and modified reports also start of Fast.ai influence

In [16]:
zipped=zip(lat_im,new_lat_im)
qs=list(zipped)

In [17]:
o_qs, n_qs = zip(*qs)

Tokenizing lists

In [18]:
ol_tok = Tokenizer.proc_all((o_qs),'en')

In [19]:
ne_tok = Tokenizer.proc_all((n_qs),'en')

In [20]:
ol_tok[4], ne_tok[4]

(['tubes',
  'and',
  'lines',
  ':',
  'none',
  '.',
  'lungs',
  '/',
  'pleura',
  ':',
  'lung',
  'volumes',
  'are',
  'slightly',
  'low',
  ',',
  'which',
  'results',
  'in',
  'mild',
  'crowding',
  'of',
  'the',
  'perihilar',
  'vasculature',
  '.',
  'there',
  'is',
  'no',
  'new',
  'focal',
  'airspace',
  'consolidation',
  '.',
  'no',
  'pleural',
  'effusion',
  'or',
  'pneumothorax',
  'is',
  'identified',
  '.',
  'a',
  'calcified',
  'nodule',
  'in',
  'the',
  'periphery',
  'of',
  'the',
  'right',
  'lung',
  'is',
  'unchanged',
  '.',
  'mediastinum',
  ':',
  'the',
  'cardiomediastinal',
  'silhouette',
  'is',
  'at',
  'the',
  'upper',
  'limits',
  'of',
  'normal',
  'for',
  'size',
  '.',
  'other',
  ':',
  'no',
  'displaced',
  'rib',
  'fracture',
  'is',
  'identified',
  ',',
  'however',
  'in',
  'the',
  'setting',
  'of',
  'chest',
  'pain',
  'if',
  'there',
  'is',
  'high',
  'clinical',
  'suspicion',
  'for',
  'rib',
  'i

# THIS IS TO GET COMMON WORDS FOR DICTIONARY

ol_tok is a list of a list of tokens for where each list is a different report so we are creating a new list of tokens from ol_tok

In [ ]:
ol_t=[]
for o in ol_tok:
    for x in o:
        ol_t.append(x)

making the list of tokens into a collections object

In [ ]:
import collections
ol_count=collections.Counter(ol_t)

using the collections method most common and putting the most commin 2000 words in a pickle 

In [ ]:
co=ol_count.most_common(2000)
pickle.dump(co, open('lat_counts.csv', 'wb'))
print(co)

These are the same steps but of the AP impressions

In [ ]:
fron_tok = Tokenizer.proc_all((front_im),'en')

In [ ]:
fron_t=[]
for o in fron_tok:
    for x in o:
        fron_t.append(x)

In [ ]:
import collections
fron_count=collections.Counter(fron_t)

In [ ]:
print(fron_count.most_common(2000))

# BACK TO FAST-AI

In [21]:
np.percentile([len(o) for o in ol_tok], 90), np.percentile([len(o) for o in ne_tok], 90)

(32.0, 36.0)

In [22]:
keep = np.array([len(o) for o in ol_tok])

In [23]:
ol_tok = np.array(ol_tok)[keep]
ne_tok = np.array(ne_tok)[keep]

Pickling ol_tok and ne_tok

In [24]:
file_name1= "1origional.pkl"
fileObject1=open(file_name1, 'wb')
pickle.dump(ol_tok,fileObject1)
file_name2= "1modified.pkl"
fileObject2=open(file_name2, 'wb')
pickle.dump(ne_tok,fileObject2)

In [25]:
pickle.dump(ol_tok, (Path('1origional').open('wb')))

In [26]:
pickle.dump(ne_tok, (Path('1modified').open('wb')))

In [27]:
ol_tok = pickle.load((Path('1origional')).open('rb'))

In [28]:
ne_tok = pickle.load((Path('1modified')).open('rb'))

In [29]:
fileObject1.close()
fileObject1=open(file_name1, 'rb')
ol_tok=pickle.load(fileObject1)
fileObject2.close()
fileObject2=open(file_name2, 'rb')
ne_tok=pickle.load(fileObject2)

Converting each token into a number and adding some pading and beginning and end of string ids

In [30]:
PATH1=Path('fastai/courses/andrea/data/translation/tmp')
def toks2ids(tok,pre):
    freq = Counter(p for o in tok for p in o)
    itos = [o for o,c in freq.most_common(40000)]
    itos.insert(0, '_bos_')
    itos.insert(1, '_pad_')
    itos.insert(2, '_eos_')
    itos.insert(3, '_unk')
    stoi = collections.defaultdict(lambda: 3, {v:k for k,v in enumerate(itos)})
    temp = None
    ids = []
    for p in tok:
        if p != None:
            temp = []
            for o in p:
                temp.append(stoi[o])
            temp += [2]
            ids.append(temp)
    ids = np.array(ids)
    print(len(ids))
    np.save(f'{pre}_ids.npy', ids)
    pickle.dump(itos, open(f'{pre}_itos.pkl', 'wb'))
    return ids,itos,stoi

If you have already done this process you nead to remove the files before doing it again

In [31]:
rm en4_ids.npy en4_itos.pkl en5_ids.npy en5_itos.pkl

rm: cannot remove 'en4_ids.npy': No such file or directory
rm: cannot remove 'en4_itos.pkl': No such file or directory
rm: cannot remove 'en5_ids.npy': No such file or directory
rm: cannot remove 'en5_itos.pkl': No such file or directory


In [32]:
ol_ids,ol_itos,ol_stoi = toks2ids(ol_tok,'en4')
ne_ids,ne_itos,ne_stoi = toks2ids(ne_tok,'en5')

100662
100662


In [33]:
def load_ids(pre):
    ids = np.load(f'{pre}_ids.npy')
    itos = pickle.load(open(f'{pre}_itos.pkl', 'rb'))
    stoi = collections.defaultdict(lambda: 3, {v:k for k,v in enumerate(itos)})
    return ids,itos,stoi

In [34]:
ol_ids,ol_itos,ol_stoi = load_ids('en4')
ne_ids,ne_itos,ne_stoi = load_ids('en5')

In [35]:
[ol_itos[o] for o in ol_ids[0]], [ne_itos[p] for p in ne_ids[0]], len(ol_itos), len(ne_itos)

(['left', 'ventricular', 'shunt', 'catheter', 'as', 'described', '.', '_eos_'],
 ['left', 'ventricular', 'shunt', 'catheter', 'as', 'described', '.', '_eos_'],
 441,
 432)

Import word vector model

In [36]:
import fastText as ft

In [37]:
en_vecs = ft.load_model(str('data/translate/wiki.en.bin'))

In [38]:
def get_vecs(lang, ft_vecs):
    vecd = {w:ft_vecs.get_word_vector(w) for w in ft_vecs.get_words()}
    pickle.dump(vecd, open('data/translate/wiki.en.pkl','wb'))
    return vecd

In [39]:
en_vecd = get_vecs('en', en_vecs)

In [40]:
en_vecd = pickle.load(open('data/translate/wiki.en.pkl','rb'))

In [41]:
ft_words = en_vecs.get_words(include_freq=True)
ft_word_dict = {k:v for k,v in zip(*ft_words)}
ft_words = sorted(ft_word_dict.keys(), key=lambda x: ft_word_dict[x])

len(ft_words)

2519370

The word vector for a comma

In [42]:
dim_en_vec = len(en_vecd[','])
dim_en_vec

300

In [43]:
en_vecs = np.stack(list(en_vecd.values()))
en_vecs.mean(),en_vecs.std()

(1.3874772e-05, 0.016063333)

In [44]:
nelen_90 = int(np.percentile([len(o) for o in ne_ids], 90))
ollen_90 = int(np.percentile([len(o) for o in ol_ids], 97))
nelen_90,ollen_90

(52, 104)

In [45]:
ne_ids_tr = np.array([o[:nelen_90] for o in ne_ids])
ol_ids_tr = np.array([o[:ollen_90] for o in ol_ids])

In [46]:
class Seq2SeqDataset(Dataset):
    def __init__(self, x, y): self.x,self.y = x,y
    def __getitem__(self, idx): return A(self.x[idx], self.y[idx])
    def __len__(self): return len(self.x)

Getting the training and validation sets

In [47]:
np.random.seed(42)
trn_keep = np.random.rand(len(ne_ids_tr))>0.1
ne_trn,ol_trn = ne_ids_tr[trn_keep],ol_ids_tr[trn_keep]
ne_val,ol_val = ne_ids_tr[~trn_keep],ol_ids_tr[~trn_keep]
len(ne_trn),len(ne_val)

(90569, 10093)

In [48]:
trn_ds = Seq2SeqDataset(ol_trn,ne_trn)
val_ds = Seq2SeqDataset(ol_val,ne_val)

batch size

In [49]:
bs=80

In [50]:
trn_samp = SortishSampler(ne_trn, key=lambda x: len(ne_trn[x]), bs=bs)
val_samp = SortSampler(ne_val, key=lambda x: len(ne_val[x]))

In [51]:
PATH= Path ('fastai/courses/andrea')

In [52]:
trn_dl = DataLoader(trn_ds, bs, transpose=True, transpose_y=True, num_workers=1, 
                    pad_idx=1, pre_pad=False, sampler=trn_samp)
val_dl = DataLoader(val_ds, int(bs*1.6), transpose=True, transpose_y=True, num_workers=1, 
                    pad_idx=1, pre_pad=False, sampler=val_samp)
md = ModelData(PATH, trn_dl, val_dl)

In [53]:
it = iter(trn_dl)
its = [next(it) for i in range(5)]
[(len(x),len(y)) for x,y in its]

[(104, 52), (104, 52), (104, 52), (32, 43), (8, 8)]

In [ ]:
Word embeddings

In [54]:
def create_emb(vecs, itos, em_sz):
    emb = nn.Embedding(len(itos), em_sz, padding_idx=1)
    wgts = emb.weight.data
    miss = []
    for i,w in enumerate(itos):
        try: wgts[i] = torch.from_numpy(vecs[w]*3)
        except: miss.append(w)
    print(len(miss),miss[5:10])
    return emb

In [55]:
nh,nl = 276,2

In [ ]:
Loss function

In [56]:
def seq2seq_loss(input, target):
    sl,bs = target.size()
    sl_in,bs_in,nc = input.size()
    if sl>sl_in: input = F.pad(input, (0,0,0,0,0,sl-sl_in))
    input = input[:sl]
    return F.cross_entropy(input.view(-1,nc), target.view(-1))#, ignore_index=1)

In [57]:
opt_fn = partial(optim.Adam, betas=(0.8, 0.99))

In [58]:
lr=3e-3

In [59]:
class Seq2SeqStepper(Stepper):
    def step(self, xs, y, epoch):
        self.m.pr_force = (10-epoch)*0.1 if epoch<10 else 0
        xtra = []
        output = self.m(*xs, y)
        if isinstance(output,tuple): output,*xtra = output
        self.opt.zero_grad()
        loss = raw_loss = self.crit(output, y)
        if self.reg_fn: loss = self.reg_fn(output, xtra, raw_loss)
        loss.backward()
        if self.clip:   # Gradient clipping
            nn.utils.clip_grad_norm(trainable_params_(self.m), self.clip)
        self.opt.step()
        return raw_loss.data[0]

In [60]:
def rand_t(*sz): return torch.randn(sz)/math.sqrt(sz[0])
def rand_p(*sz): return nn.Parameter(rand_t(*sz))

In [61]:
class Seq2SeqRNN_All(nn.Module):
    def __init__(self, vecs_enc, itos_enc, em_sz_enc, vecs_dec, itos_dec, em_sz_dec, nh, out_sl, nl=2):
        super().__init__()
        self.emb_enc = create_emb(vecs_enc, itos_enc, em_sz_enc)
        self.nl,self.nh,self.out_sl = nl,nh,out_sl
        self.gru_enc = nn.GRU(em_sz_enc, nh, num_layers=nl, dropout=0.25, bidirectional=True)
        self.out_enc = nn.Linear(nh*2, em_sz_dec, bias=False)
        self.drop_enc = nn.Dropout(0.25)
        self.emb_dec = create_emb(vecs_dec, itos_dec, em_sz_dec)
        self.gru_dec = nn.GRU(em_sz_dec, em_sz_dec, num_layers=nl, dropout=0.1)
        self.emb_enc_drop = nn.Dropout(0.15)
        self.out_drop = nn.Dropout(0.35)
        self.out = nn.Linear(em_sz_dec, len(itos_dec))
        self.out.weight.data = self.emb_dec.weight.data

        self.W1 = rand_p(nh*2, em_sz_dec)
        self.l2 = nn.Linear(em_sz_dec, em_sz_dec)
        self.l3 = nn.Linear(em_sz_dec+nh*2, em_sz_dec)
        self.V = rand_p(em_sz_dec)

    def forward(self, inp, y=None):
        sl,bs = inp.size()
        h = self.initHidden(bs)
        emb = self.emb_enc_drop(self.emb_enc(inp))
        enc_out, h = self.gru_enc(emb, h)
        h = h.view(2,2,bs,-1).permute(0,2,1,3).contiguous().view(2,bs,-1)
        h = self.out_enc(self.drop_enc(h))

        dec_inp = V(torch.zeros(bs).long())
        res,attns = [],[]
        w1e = enc_out @ self.W1
        for i in range(self.out_sl):
            w2h = self.l2(h[-1])
            u = F.tanh(w1e + w2h)
            a = F.softmax(u @ self.V, 0)
            attns.append(a)
            Xa = (a.unsqueeze(2) * enc_out).sum(0)
            emb = self.emb_dec(dec_inp)
            wgt_enc = self.l3(torch.cat([emb, Xa], 1))
            
            outp, h = self.gru_dec(wgt_enc.unsqueeze(0), h)
            outp = self.out(self.out_drop(outp[0]))
            res.append(outp)
            dec_inp = V(outp.data.max(1)[1])
            if (dec_inp==1).all(): break
            if (y is not None) and (random.random()<self.pr_force):
                if i>=len(y): break
                dec_inp = y[i]
        return torch.stack(res)

    def initHidden(self, bs): return V(torch.zeros(self.nl*2, bs, self.nh))

In [62]:
rnn = Seq2SeqRNN_All(en_vecd, ol_itos, dim_en_vec, en_vecd, ne_itos, dim_en_vec, nh, nelen_90)
learn = RNN_Learner(md, SingleModel(to_gpu(rnn)), opt_fn=opt_fn)
learn.crit = seq2seq_loss

37 ['2015', '9', 'midlung', 'r.', '20']
36 ['2015', '9', 'midlung', 'r.', '20']


Learning rate

In [63]:
lr=2e-3

The training

In [64]:
learn.fit(lr, 1, cycle_len=10, use_clr=(20,10), stepper=Seq2SeqStepper)

epoch      trn_loss   val_loss                                 
    0      0.214791   2.272808  
    1      0.13531    1.069795                                 
    2      0.075025   0.385344                                  
    3      0.045793   0.097252                                  
    4      0.038328   0.037351                                  
    5      0.031759   0.017906                                  
    6      0.026524   0.020483                                  
    7      0.050914   0.010068                                  
    8      0.086006   0.01084                                   
    9      0.046962   0.007734                                  



[array([0.00773])]

Some printed output, top result is the original report, next is the goal translation from the word replacement, last is the ai training generated translation

In [65]:
x,y = next(iter(val_dl))
probs = learn.model(V(x))
preds = to_np(probs.max(2)[1])

for i in range(120,128):
    print(' '.join([ol_itos[o] for o in x[:,i] if o != 1]))
    print(' '.join([ne_itos[o] for o in y[:,i] if o != 1]))
    print(' '.join([ne_itos[o] for o in preds[:,i] if o!=1]))
    print()

tubes and lines : none . lungs / pleura : lung volumes are slightly low , which results in mild crowding of the perihilar vasculature . there is no new focal airspace consolidation . no pleural effusion or pneumothorax is identified . a calcified nodule in the periphery of the right lung is unchanged . mediastinum : the cardiomediastinal silhouette is at the upper limits of normal for size . other : no displaced rib fracture is identified , however in the setting of chest pain if there is high clinical suspicion for rib injury , a dedicated rib series is recommended . _eos_
tubes and lines : none . lungs / pleura : lung volumes are slightly low , which results in mild crowding of the perihilar vessels . there is no new focal airspace consolidation . no fluid around the lungs or partial collapse of lung is identified . a calcified nodule in the
tubes and lines : none . lungs / pleura : lung volumes are slightly low , which results in mild crowding of the perihilar vessels . there is no 

# For MT-ComparEval File Prep

Making two text files from the original text and the generated ai translation

In [ ]:
x,y = next(iter(val_dl))
probs = learn.model(V(x))
preds = to_np(probs.max(2)[1])
s=open('source2.txt', 'w')
r=open('ref2.txt', 'w')
for i in range(1,128):
    s.write(' '.join([ol_itos[o] for o in x[:,i] if o != 1]))
    s.write("\n")
    r.write(' '.join([ne_itos[o] for o in preds[:,i] if o!=1]))
    r.write("\n")